In [1]:
import numpy as np
import csv
from scipy import optimize as op
import math
import copy

In [2]:
#input basedcurves and observed TASs
path="E:/"

observedpath=path+"observed.csv"
basecurvepath=path+"basecurve.csv"

observed=np.array(list(csv.reader(open(observedpath,'r'))))
basecurve=np.array(list(csv.reader(open(basecurvepath,'r'))))


In [3]:
#cosine similarity
def calcos(A,B):
    N=len(A)
    suma2=0
    sumb2=0
    sumab=0
    for i in range(N):
        suma2+=float(A[i])*float(A[i])
        sumb2+=float(B[i])*float(B[i])
        sumab+=float(A[i])*float(B[i])
    if(suma2==0 or sumb2==0):
        return -2
    r=sumab/(math.sqrt(suma2)*math.sqrt(sumb2))
    return r

In [4]:
#optimize
#bases and tmpcurve are global variables
def similarity(X):
    L=len(tmpcurve)
    N=len(X)
    newcurve=np.zeros(L)
    for i in range(N):
        for j in range(L):
            newcurve[j]+=float(X[i])*float(bases[i][j])
    result=calcos(newcurve,tmpcurve)
    return 1-result

def constraint(X):
    return sum(X)-1

In [5]:
#other results
#mixing index
def calmix(weights):
    result=0
    for i in range(len(weights)):
        if(float(weights[i])==0):
            continue
        result+=(-1)*float(weights[i])*math.log(float(weights[i]))
    return result

#dynamic activity proportion
def caldynamic(weights,bases):
    L=len(bases[0])
    M=len(weights)
    total=np.zeros(L)
    for i in range(M):
        for j in range(L):
            total[j]+=float(weights[i])*float(bases[i][j])
    result=[]
    for j in range(L):
        tmp=[]
        for i in range(M):
            tmpro=float(weights[i])*float(bases[i][j])/total[j]
            tmp.append(tmpro)
        result.append(tmp)
    return np.array(result)

In [6]:
#decomposition
decomposed=np.array(observed)
bases=np.array(basecurve)
N=len(decomposed)
M=len(bases)

#>=0,<=1
b=(0.0,1.0)
bnds=[]
#initialization weights
x0=[]
for i in range(M):
    tmp=(0.0,1.0)
    bnds.append(tmp)
    x0.append(1.0/M)
bnds=tuple(bnds)


cons=[{'type' : 'eq' , 'fun' : constraint}]

for j in range(N):
    tmpcurve=copy.deepcopy(decomposed[j])
    sol=op.minimize(similarity,x0,method='SLSQP',bounds=bnds,constraints=cons)
    composition=np.array(sol['x'])
    dynamicpro=caldynamic(composition,bases)
    mixingindex=calmix(composition)
    print(composition,mixingindex)
    if(j==10):
        print(dynamicpro)
    print("-------------------------------")



[  0.00000000e+00   9.29515522e-01   0.00000000e+00   7.04844780e-02
   6.26402248e-17] 0.254890342183164
-------------------------------
[  0.00000000e+00   1.00000000e+00   4.67659656e-17   6.93889390e-18
   7.88823115e-17] 4.957459789324139e-15
-------------------------------
[  3.70999641e-01   6.29000359e-01   0.00000000e+00   8.67361738e-17
   1.24303779e-16] 0.6594855639200745
-------------------------------
[  0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   4.86823289e-17] 1.8285674506587357e-15
-------------------------------
[  0.00000000e+00   6.26826109e-01   8.85630418e-18   1.06278565e-01
   2.66895325e-01] 0.8835672430439206
-------------------------------
[ 0.          0.41317815  0.06223192  0.          0.52458993] 0.8764425530629117
-------------------------------
[  0.00000000e+00   6.84084203e-01   3.15915797e-01   0.00000000e+00
   1.60426812e-18] 0.6237524842820085
-------------------------------
[  4.16333634e-17   0.00000000e+00   6.93889390